# Merging diffrent dataset to and creating the pipeline

In [1]:
import pandas as pd
weather_df = pd.read_csv('weatherHistory.csv')
weather_df['Formatted Date'] = pd.to_datetime(weather_df['Formatted Date'], format='%Y-%m-%d %H:%M:%S.%f %z')

### Load Dataset

In [2]:
# Load datasets
traffic_df = pd.read_csv('traffic.csv', parse_dates=['DateTime'])
weather_df = pd.read_csv('weatherHistory.csv', parse_dates=['Formatted Date'])
events_df = pd.read_csv('festivals_kaggle.csv', parse_dates=['Date'])
weather_df = pd.read_csv('weatherHistory.csv')

# Convert to datetime and handle irregular dates dates

In [6]:
# Assuming weather_df['Formatted Date'] is a string representation of datetime
weather_df['Formatted Date'] = pd.to_datetime(weather_df['Formatted Date'], utc=True)

# Now convert to naive datetime (remove timezone information)
weather_df['Formatted Date'] = weather_df['Formatted Date'].dt.tz_localize(None)

def safe_to_datetime(series):
    return pd.to_datetime(series, errors='coerce')

# Apply safe_to_datetime and handle NaT values
traffic_df['DateTime'] = safe_to_datetime(traffic_df['DateTime'])
traffic_df['DateTime'] = traffic_df['DateTime'].dt.tz_localize(None)

weather_df['Formatted Date'] = safe_to_datetime(weather_df['Formatted Date'])
weather_df.loc[weather_df['Formatted Date'].notna(), 'Formatted Date'] = weather_df.loc[weather_df['Formatted Date'].notna(), 'Formatted Date'].dt.tz_localize(None)

events_df['Date'] = safe_to_datetime(events_df['Date'])
events_df['Date'] = events_df['Date'].dt.tz_localize(None)


C:\Users\amira\AppData\Local\Temp\ipykernel_15040\122036971.py:15: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  weather_df.loc[weather_df['Formatted Date'].notna(), 'Formatted Date'] = weather_df.loc[weather_df['Formatted Date'].notna(), 'Formatted Date'].dt.tz_localize(None)


# Handling missing values 

In [8]:
traffic_df = traffic_df.dropna(subset=['DateTime'])
weather_df = weather_df.dropna(subset=['Formatted Date'])
events_df = events_df.dropna(subset=['Date'])

In [9]:
#Converting all dates to the same format

In [10]:
traffic_df['DateTime'] = traffic_df['DateTime'].dt.strftime('%Y-%m-%d %H:%M:%S')
weather_df['Formatted Date'] = weather_df['Formatted Date'].dt.strftime('%Y-%m-%d %H:%M:%S')
events_df['Date'] = events_df['Date'].dt.strftime('%Y-%m-%d %H:%M:%S')

In [11]:
traffic_df['DateTime'] = pd.to_datetime(traffic_df['DateTime'])
weather_df['Formatted Date'] = pd.to_datetime(weather_df['Formatted Date'])
events_df['Date'] = pd.to_datetime(events_df['Date'])

# replacing the name of all 3 colums to single one for easy mearging

In [12]:
weather_df.rename(columns={'Formatted Date': 'DateTime'}, inplace=True)
events_df.rename(columns={'Date': 'DateTime'}, inplace=True)

# Merging by common value that is hour in all three dataset

In [13]:
traffic_df['DateTime'] = traffic_df['DateTime'].dt.floor('H')
weather_df['DateTime'] = weather_df['DateTime'].dt.floor('H')
events_df['DateTime'] = events_df['DateTime'].dt.floor('H')

In [14]:
merged_df = pd.merge(traffic_df, weather_df, on='DateTime', how='outer')

In [15]:
merged_df = pd.merge(merged_df, events_df, on='DateTime', how='outer')

In [16]:
merged_df.fillna(method='ffill', inplace=True)

# final cleaning of dataset by droping the unwanted colums

In [23]:
merged_df.head()
merged_df.drop(columns=['Id','Day','Year','Daily Summary','Id'],inplace=True)

In [24]:
merged_df.to_csv('merged_data_AamirAdnan.csv', index=False)

In [25]:
merged_df.head()

,DateTime,Junction,Vehicles,ID,Summary,Precip Type,Temperature (C),Apparent Temperature (C),Humidity,Wind Speed (km/h),Wind Bearing (degrees),Visibility (km),Loud Cover,Pressure (millibars),Festival name
0,2015-01-11 00:00:00,1.0,15.0,2.015110e+10,Partly Cloudy,rain,1.483333,-1.711111,0.92,10.7709,201.0,8.6457,0.0,1010.86,NaN
1,2015-01-11 00:00:00,2.0,6.0,2.015110e+10,Partly Cloudy,rain,1.483333,-1.711111,0.92,10.7709,201.0,8.6457,0.0,1010.86,NaN
2,2015-01-11 00:00:00,3.0,9.0,2.015110e+10,Partly Cloudy,rain,1.483333,-1.711111,0.92,10.7709,201.0,8.6457,0.0,1010.86,NaN
3,2015-01-11 01:00:00,1.0,13.0,2.015110e+10,Partly Cloudy,rain,1.911111,-1.188889,0.92,10.7387,184.0,7.4865,0.0,1010.78,NaN
4,2015-01-11 01:00:00,2.0,6.0,2.015110e+10,Partly Cloudy,rain,1.911111,-1.188889,0.92,10.7387,184.0,7.4865,0.0,1010.78,NaN
